## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Mar. 2nd Mar. 9th


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,March-4-2025,Condo/Co-op,300 NE 20th St #109,Boca Raton,FL,33431.0,125000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/300-NE-20...,Beaches MLS,F10459945,N,Y,26.366103,-80.081778


In [28]:
print('Input county name:')
county = input()

Input county name:
Palm Beach County


In [29]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [30]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [31]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [32]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-03-03    29
2025-03-07    24
2025-03-04    23
2025-03-05    18
2025-03-06    16
2025-03-08     1
Name: count, dtype: int64

In [33]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [34]:
df_top_ten = df2.head(10)

In [35]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
50,PAST SALE,2025-03-07,Condo/Co-op,2494 S Ocean Blvd Unit L-3,Boca Raton,FL,33432.0,5450000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Beaches MLS,RX-11042965,N,Y,26.327226,-80.074809
111,PAST SALE,2025-03-03,Condo/Co-op,622 N Flagler Dr #803,West Palm Beach,FL,33401.0,4900000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/622-...,Beaches MLS,RX-10876816,N,Y,26.719123,-80.049156
40,PAST SALE,2025-03-03,Condo/Co-op,3730 N Ocean 4b Dr Unit 4b,Riviera Beach,FL,33404.0,4200000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Beaches MLS,RX-11048530,N,Y,26.791736,-80.033375
96,PAST SALE,2025-03-07,Condo/Co-op,2323 Lake Dr Unit 302,Singer Island,FL,33404.0,3400000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/2323...,Beaches MLS,RX-11004952,N,Y,26.781953,-80.039438
87,PAST SALE,2025-03-07,Condo/Co-op,1400 S Ocean Blvd #604,Boca Raton,FL,33432.0,2519400.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Beaches MLS,RX-11018243,N,Y,26.331256,-80.074321
81,PAST SALE,2025-03-07,Condo/Co-op,550 S Ocean Blvd #401,Boca Raton,FL,33432.0,1580000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11026297,N,Y,26.344431,-80.070692
104,PAST SALE,2025-03-04,Condo/Co-op,2660 S Ocean Blvd Unit 702n,Palm Beach,FL,33480.0,1500000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-10975482,N,Y,26.623992,-80.037641
36,PAST SALE,2025-03-03,Condo/Co-op,500 SE Mizner Blvd Unit A902,Boca Raton,FL,33432.0,1460000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/500-SE-Mi...,Beaches MLS,RX-11040277,N,Y,26.345677,-80.084004
56,PAST SALE,2025-03-03,Condo/Co-op,2917 S Ocean Blvd #1002,Highland Beach,FL,33487.0,1325000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/2917-...,Beaches MLS,RX-11043172,N,Y,26.422028,-80.063634
4,PAST SALE,2025-03-05,Condo/Co-op,362 Chambord Ter,Palm Beach Gardens,FL,33410.0,1140000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/3...,Beaches MLS,RX-11051333,N,Y,26.865330,-80.094483


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [36]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [37]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [38]:
response_df = pd.DataFrame(response_list)

In [39]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [40]:
df_top_ten = merged_df

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-03-07,Condo/Co-op,2494 S Ocean Blvd Unit L-3,Boca Raton,FL,33432.0,5450000.0,4.0,4.5,...,-80.074809,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Catherine Kohn,One World Properties,None,None
1,PAST SALE,2025-03-03,Condo/Co-op,622 N Flagler Dr #803,West Palm Beach,FL,33401.0,4900000.0,3.0,3.5,...,-80.049156,https://www.redfin.com/FL/West-Palm-Beach/622-...,Margit Brandt,"Premier Estate Properties, Inc",None,None,Adam Levitt,The Corcoran Group,None,None
2,PAST SALE,2025-03-03,Condo/Co-op,3730 N Ocean 4b Dr Unit 4b,Riviera Beach,FL,33404.0,4200000.0,4.0,4.5,...,-80.033375,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Brian Jacobs,"Network Property Group, LLC",None,None,Brian Jacobs,"Network Property Group, LLC",None,None
3,PAST SALE,2025-03-07,Condo/Co-op,2323 Lake Dr Unit 302,Singer Island,FL,33404.0,3400000.0,3.0,3.5,...,-80.039438,https://www.redfin.com/FL/West-Palm-Beach/2323...,Meike MacGregor,Compass Florida LLC,Kevin Spina,The Keyes Company (PBG),Troy Maleyko,Illustrated Properties LLC (Jupiter),None,None
4,PAST SALE,2025-03-07,Condo/Co-op,1400 S Ocean Blvd #604,Boca Raton,FL,33432.0,2519400.0,2.0,2.5,...,-80.074321,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,David Roberts,Sanctuary Realty LLC,Kelly Brooks,Sanctuary Realty LLC,Bonnie Heatzig,Compass Florida LLC,Mary Mangines,Compass Florida LLC
5,PAST SALE,2025-03-07,Condo/Co-op,550 S Ocean Blvd #401,Boca Raton,FL,33432.0,1580000.0,2.0,2.0,...,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Kim Schlinck,Lang Realty/BR,None,None
6,PAST SALE,2025-03-04,Condo/Co-op,2660 S Ocean Blvd Unit 702n,Palm Beach,FL,33480.0,1500000.0,2.0,2.5,...,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Cathy Casella,Brown Harris Stevens of PB,Lauren Sbordone,Brown Harris Stevens of PB,Cathy Casella,Brown Harris Stevens of PB,None,None
7,PAST SALE,2025-03-03,Condo/Co-op,500 SE Mizner Blvd Unit A902,Boca Raton,FL,33432.0,1460000.0,2.0,2.5,...,-80.084004,https://www.redfin.com/FL/Boca-Raton/500-SE-Mi...,Caryn Nadler,"Premier Estate Properties, Inc",None,None,Caryn Nadler,"Premier Estate Properties, Inc",None,None
8,PAST SALE,2025-03-03,Condo/Co-op,2917 S Ocean Blvd #1002,Highland Beach,FL,33487.0,1325000.0,2.0,2.0,...,-80.063634,https://www.redfin.com/FL/Highland-Beach/2917-...,Torm Martinson,Campbell&Rosemurgy Real Estate,None,None,Eric Levy,Coldwell Banker,None,None
9,PAST SALE,2025-03-05,Condo/Co-op,362 Chambord Ter,Palm Beach Gardens,FL,33410.0,1140000.0,2.0,2.5,...,-80.094483,https://www.redfin.com/FL/Palm-Beach-Gardens/3...,Sylvia Isaacs,Echo Fine Properties,Jeffrey Lichtenstein,Echo Fine Properties,Chris Burke,Echo Fine Properties,Jeffrey Lichtenstein,Echo Fine Properties


## Current Week's Values

In [42]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

111


In [43]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$513,901


In [44]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$311


In [45]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$57,042,970


In [47]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PDF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PDF  Last Week Total Dollar Volume
0          111              513901          311                     57042970.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [48]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PDF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
129
-----------
Input Previous Week Condo Average Sales Price:
617735
-----------
Input Previous Week Condo Average PSF:
366
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
79687754.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [49]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [50]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [51]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [52]:
pd.set_option('display.max_columns',None)

In [53]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-07,Condo/Co-op,2494 S Ocean Blvd Unit L-3,Boca Raton,FL,33432.0,5450000.0,4.0,4.5,Aragon Condo,4718.0,NaN,1995.0,NaN,1155.0,7524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Beaches MLS,RX-11042965,N,Y,26.327226,-80.074809,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Catherine Kohn,One World Properties,None,None,1,orange


In [54]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [55]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-07,Condo/Co-op,2494 S Ocean Blvd Unit L-3,Boca Raton,FL,33432.0,5450000.0,4.0,4.5,Aragon Condo,4718.0,NaN,1995.0,NaN,1155.0,7524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Beaches MLS,RX-11042965,N,Y,26.327226,-80.074809,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Catherine Kohn,One World Properties,None,None,1,orange
1,PAST SALE,2025-03-03,Condo/Co-op,622 N Flagler Dr #803,West Palm Beach,FL,33401.0,4900000.0,3.0,3.5,One Watermark Place OF The Palm Beaches,4035.0,NaN,2003.0,NaN,1214.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/622-...,Beaches MLS,RX-10876816,N,Y,26.719123,-80.049156,https://www.redfin.com/FL/West-Palm-Beach/622-...,Margit Brandt,"Premier Estate Properties, Inc",None,None,Adam Levitt,The Corcoran Group,None,None,2,blue
2,PAST SALE,2025-03-03,Condo/Co-op,3730 N Ocean 4b Dr Unit 4b,Riviera Beach,FL,33404.0,4200000.0,4.0,4.5,Vistablue Singer Island Condo,3642.0,NaN,2018.0,NaN,1153.0,4329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Beaches MLS,RX-11048530,N,Y,26.791736,-80.033375,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Brian Jacobs,"Network Property Group, LLC",None,None,Brian Jacobs,"Network Property Group, LLC",None,None,3,blue
3,PAST SALE,2025-03-07,Condo/Co-op,2323 Lake Dr Unit 302,Singer Island,FL,33404.0,3400000.0,3.0,3.5,Palm Beach Shores,2589.0,NaN,2024.0,NaN,1313.0,2097.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/2323...,Beaches MLS,RX-11004952,N,Y,26.781953,-80.039438,https://www.redfin.com/FL/West-Palm-Beach/2323...,Meike MacGregor,Compass Florida LLC,Kevin Spina,The Keyes Company (PBG),Troy Maleyko,Illustrated Properties LLC (Jupiter),None,None,4,blue
4,PAST SALE,2025-03-07,Condo/Co-op,1400 S Ocean Blvd #604,Boca Raton,FL,33432.0,2519400.0,2.0,2.5,Addison ON The Ocean IN Boca Raton Condo,2156.0,NaN,1985.0,NaN,1169.0,3491.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Beaches MLS,RX-11018243,N,Y,26.331256,-80.074321,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,David Roberts,Sanctuary Realty LLC,Kelly Brooks,Sanctuary Realty LLC,Bonnie Heatzig,Compass Florida LLC,Mary Mangines,Compass Florida LLC,5,blue
5,PAST SALE,2025-03-07,Condo/Co-op,550 S Ocean Blvd #401,Boca Raton,FL,33432.0,1580000.0,2.0,2.0,Chalfonte Condo,1479.0,NaN,1974.0,NaN,1068.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11026297,N,Y,26.344431,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Kim Schlinck,Lang Realty/BR,None,None,6,blue
6,PAST SALE,2025-03-04,Condo/Co-op,2660 S Ocean Blvd Unit 702n,Palm Beach,FL,33480.0,1500000.0,2.0,2.5,Beach Point Condo,1900.0,NaN,1979.0,NaN,789.0,2308.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-10975482,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Cathy Casella,Brown Harris Stevens of PB,Lauren Sbordone,Brown Harris Stevens of PB,Cathy Casella,Brown Harris Stevens of PB,None,None,7,blue
7,PAST SALE,2025-03-03,Condo/Co-op,500 SE Mizner Blvd Unit A902,Boca Raton,FL,33432.0,1460000.0,2.0,2.5,Townsend Place Condominium,2005.0,NaN,1998.0,NaN,728.0,2512.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/500-SE-Mi...,Beaches MLS,RX-11040277,N,Y,26.345677,-80.084004,https://www.redfin.com/FL/Boca-Raton/500-SE-Mi...,Caryn Nadler,"Premier Estate Properties, Inc",None,None,Caryn Nadler,"Premier Estate Pr

In [56]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [57]:
m.save('index.html')

## Data snagger

In [58]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [59]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-07,Condo/Co-op,2494 S Ocean Blvd Unit L-3,Boca Raton,FL,33432.0,5450000.0,4.0,4.5,Aragon Condo,4718.0,NaN,1995.0,NaN,1155.0,7524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Beaches MLS,RX-11042965,N,Y,26.327226,-80.074809,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Catherine Kohn,One World Properties,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [60]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_03102024


# CREATE TEMPLATE 

In [61]:
muni_set = set(df_top_ten['CITY'])

In [62]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [63]:
df_top_ten.reset_index(inplace=True,drop=True)

In [64]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [65]:
top_sale

'Aragon Condo at 2494 S Ocean Blvd Unit L-3 in Boca Raton'

In [66]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [67]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [68]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-07,Condo/Co-op,2494 S Ocean Blvd Unit L-3,Boca Raton,FL,33432.0,5450000.0,4.0,4.5,Aragon Condo,4718.0,NaN,1995.0,NaN,1155.0,7524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Beaches MLS,RX-11042965,N,Y,26.327226,-80.074809,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Catherine Kohn,One World Properties,None,None,1,orange,Aragon Condo at 2494 S Ocean Blvd Unit L-3 in ...
1,PAST SALE,2025-03-03,Condo/Co-op,622 N Flagler Dr #803,West Palm Beach,FL,33401.0,4900000.0,3.0,3.5,One Watermark Place OF The Palm Beaches,4035.0,NaN,2003.0,NaN,1214.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/622-...,Beaches MLS,RX-10876816,N,Y,26.719123,-80.049156,https://www.redfin.com/FL/West-Palm-Beach/622-...,Margit Brandt,"Premier Estate Properties, Inc",None,None,Adam Levitt,The Corcoran Group,None,None,2,blue,One Watermark Place OF The Palm Beaches at 622...
2,PAST SALE,2025-03-03,Condo/Co-op,3730 N Ocean 4b Dr Unit 4b,Riviera Beach,FL,33404.0,4200000.0,4.0,4.5,Vistablue Singer Island Condo,3642.0,NaN,2018.0,NaN,1153.0,4329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Beaches MLS,RX-11048530,N,Y,26.791736,-80.033375,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Brian Jacobs,"Network Property Group, LLC",None,None,Brian Jacobs,"Network Property Group, LLC",None,None,3,blue,Vistablue Singer Island Condo at 3730 N Ocean ...
3,PAST SALE,2025-03-07,Condo/Co-op,2323 Lake Dr Unit 302,Singer Island,FL,33404.0,3400000.0,3.0,3.5,Palm Beach Shores,2589.0,NaN,2024.0,NaN,1313.0,2097.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/2323...,Beaches MLS,RX-11004952,N,Y,26.781953,-80.039438,https://www.redfin.com/FL/West-Palm-Beach/2323...,Meike MacGregor,Compass Florida LLC,Kevin Spina,The Keyes Company (PBG),Troy Maleyko,Illustrated Properties LLC (Jupiter),None,None,4,blue,Palm Beach Shores at 2323 Lake Dr Unit 302 in ...
4,PAST SALE,2025-03-07,Condo/Co-op,1400 S Ocean Blvd #604,Boca Raton,FL,33432.0,2519400.0,2.0,2.5,Addison ON The Ocean IN Boca Raton Condo,2156.0,NaN,1985.0,NaN,1169.0,3491.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Beaches MLS,RX-11018243,N,Y,26.331256,-80.074321,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,David Roberts,Sanctuary Realty LLC,Kelly Brooks,Sanctuary Realty LLC,Bonnie Heatzig,Compass Florida LLC,Mary Mangines,Compass Florida LLC,5,blue,Addison ON The Ocean IN Boca Raton Condo at 14...
5,PAST SALE,2025-03-07,Condo/Co-op,550 S Ocean Blvd #401,Boca Raton,FL,33432.0,1580000.0,2.0,2.0,Chalfonte Condo,1479.0,NaN,1974.0,NaN,1068.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11026297,N,Y,26.344431,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Kim Schlinck,Lang Realty/BR,None,None,6,blue,Chalfonte Condo at 550 S Ocean Blvd #401 in Bo...
6,PAST SALE,2025-03-04,Condo/Co-op,2660 S Ocean Blvd Unit 702n,Palm Beach,FL,33480.0,1500000.0,2.0,2.5,Beach Point Condo,1900.0,NaN,1979.0,NaN,789.0,2308.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-10975482,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Cathy Casella,Brown Harris Stevens of PB,Lauren Sbordone,Brown Harris Stevens of PB,Cathy Casella,Brown Harris Stevens of PB,None,None,7,blue,Beach Point Condo at 2660 S Ocean Blvd Unit 70...
7,PAST SALE,2025-03-03,Condo/Co-op,500 SE Mizner Blvd U

In [69]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Aragon Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $1,140,000 to $5,450,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Boca Raton, Palm Beach, Singer Island, West Palm Beach, Highland Beach, Riviera Beach, Palm Beach Gardens, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 111 condo sales totaling $57,042,970 million from Mar. 2nd to Mar. 9th. The previous week, brokers closed 129 condo sales totaling $79,687,754.

Last week’s

In [70]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [71]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [72]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [73]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-07,Condo/Co-op,2494 S Ocean Blvd Unit L-3,Boca Raton,FL,33432.0,5450000.0,4.0,4.5,Aragon Condo,4718.0,NaN,1995.0,NaN,1155.0,7524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Beaches MLS,RX-11042965,N,Y,26.327226,-80.074809,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Catherine Kohn,One World Properties,None,None,1,orange,Aragon Condo at 2494 S Ocean Blvd Unit L-3 in ...
1,PAST SALE,2025-03-03,Condo/Co-op,622 N Flagler Dr #803,West Palm Beach,FL,33401.0,4900000.0,3.0,3.5,One Watermark Place OF The Palm Beaches,4035.0,NaN,2003.0,NaN,1214.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/622-...,Beaches MLS,RX-10876816,N,Y,26.719123,-80.049156,https://www.redfin.com/FL/West-Palm-Beach/622-...,Margit Brandt,"Premier Estate Properties, Inc",None,None,Adam Levitt,The Corcoran Group,None,None,2,blue,One Watermark Place OF The Palm Beaches at 622...
2,PAST SALE,2025-03-03,Condo/Co-op,3730 N Ocean 4b Dr Unit 4b,Riviera Beach,FL,33404.0,4200000.0,4.0,4.5,Vistablue Singer Island Condo,3642.0,NaN,2018.0,NaN,1153.0,4329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Beaches MLS,RX-11048530,N,Y,26.791736,-80.033375,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Brian Jacobs,"Network Property Group, LLC",None,None,Brian Jacobs,"Network Property Group, LLC",None,None,3,blue,Vistablue Singer Island Condo at 3730 N Ocean ...
3,PAST SALE,2025-03-07,Condo/Co-op,2323 Lake Dr Unit 302,Singer Island,FL,33404.0,3400000.0,3.0,3.5,Palm Beach Shores,2589.0,NaN,2024.0,NaN,1313.0,2097.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/2323...,Beaches MLS,RX-11004952,N,Y,26.781953,-80.039438,https://www.redfin.com/FL/West-Palm-Beach/2323...,Meike MacGregor,Compass Florida LLC,Kevin Spina,The Keyes Company (PBG),Troy Maleyko,Illustrated Properties LLC (Jupiter),None,None,4,blue,Palm Beach Shores at 2323 Lake Dr Unit 302 in ...
4,PAST SALE,2025-03-07,Condo/Co-op,1400 S Ocean Blvd #604,Boca Raton,FL,33432.0,2519400.0,2.0,2.5,Addison ON The Ocean IN Boca Raton Condo,2156.0,NaN,1985.0,NaN,1169.0,3491.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Beaches MLS,RX-11018243,N,Y,26.331256,-80.074321,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,David Roberts,Sanctuary Realty LLC,Kelly Brooks,Sanctuary Realty LLC,Bonnie Heatzig,Compass Florida LLC,Mary Mangines,Compass Florida LLC,5,blue,Addison ON The Ocean IN Boca Raton Condo at 14...
5,PAST SALE,2025-03-07,Condo/Co-op,550 S Ocean Blvd #401,Boca Raton,FL,33432.0,1580000.0,2.0,2.0,Chalfonte Condo,1479.0,NaN,1974.0,NaN,1068.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11026297,N,Y,26.344431,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Kim Schlinck,Lang Realty/BR,None,None,6,blue,Chalfonte Condo at 550 S Ocean Blvd #401 in Bo...
6,PAST SALE,2025-03-04,Condo/Co-op,2660 S Ocean Blvd Unit 702n,Palm Beach,FL,33480.0,1500000.0,2.0,2.5,Beach Point Condo,1900.0,NaN,1979.0,NaN,789.0,2308.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-10975482,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Cathy Casella,Brown Harris Stevens of PB,Lauren Sbordone,Brown Harris Stevens of PB,Cathy Casella,Brown Harris Stevens of PB,None,None,7,blue,Beach Point Condo at 2660 S Ocean Blvd Unit 70...
7,PAST SALE,2025-03-03,Condo/Co-op,500 SE Mizner Blvd U

In [74]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Aragon Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $1,140,000 to $5,450,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Boca Raton, Palm Beach, Singer Island, West Palm Beach, Highland Beach, Riviera Beach, Palm Beach Gardens, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 111 condo sales totaling $57,042,970 million from Mar. 2nd to Mar. 9th. The previous week, brokers closed 129 condo sales totaling $79,687,754.

Last week’s

In [82]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Boca-Raton/2494-S-Ocean-Blvd-33432/unit-L3/home/42335476


In [83]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/West-Palm-Beach/622-N-Flagler-Dr-33401/unit-803/home/42592577


In [84]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Palm-Beach-Gardens/362-Chambord-Ter-33410/home/42588870


In [85]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Highland-Beach/2917-S-Ocean-Blvd-33487/unit-1002/home/42386891


In [86]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/West-Palm-Beach/2323-Lake-Dr-33404/unit-302/home/190633410


In [87]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Palm-Beach-Gardens/362-Chambord-Ter-33410/home/42588870


In [88]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-07,Condo/Co-op,2494 S Ocean Blvd Unit L-3,Boca Raton,FL,33432.0,5450000.0,4.0,4.5,Aragon Condo,4718.0,NaN,1995.0,NaN,1155.0,7524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Beaches MLS,RX-11042965,N,Y,26.327226,-80.074809,https://www.redfin.com/FL/Boca-Raton/2494-S-Oc...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Catherine Kohn,One World Properties,None,None,1,orange,Aragon Condo at 2494 S Ocean Blvd Unit L-3 in ...
1,PAST SALE,2025-03-03,Condo/Co-op,622 N Flagler Dr #803,West Palm Beach,FL,33401.0,4900000.0,3.0,3.5,One Watermark Place OF The Palm Beaches,4035.0,NaN,2003.0,NaN,1214.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/622-...,Beaches MLS,RX-10876816,N,Y,26.719123,-80.049156,https://www.redfin.com/FL/West-Palm-Beach/622-...,Margit Brandt,"Premier Estate Properties, Inc",None,None,Adam Levitt,The Corcoran Group,None,None,2,blue,One Watermark Place OF The Palm Beaches at 622...
2,PAST SALE,2025-03-03,Condo/Co-op,3730 N Ocean 4b Dr Unit 4b,Riviera Beach,FL,33404.0,4200000.0,4.0,4.5,Vistablue Singer Island Condo,3642.0,NaN,2018.0,NaN,1153.0,4329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Beaches MLS,RX-11048530,N,Y,26.791736,-80.033375,https://www.redfin.com/FL/Riviera-Beach/3730-N...,Brian Jacobs,"Network Property Group, LLC",None,None,Brian Jacobs,"Network Property Group, LLC",None,None,3,blue,Vistablue Singer Island Condo at 3730 N Ocean ...
3,PAST SALE,2025-03-07,Condo/Co-op,2323 Lake Dr Unit 302,Singer Island,FL,33404.0,3400000.0,3.0,3.5,Palm Beach Shores,2589.0,NaN,2024.0,NaN,1313.0,2097.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/2323...,Beaches MLS,RX-11004952,N,Y,26.781953,-80.039438,https://www.redfin.com/FL/West-Palm-Beach/2323...,Meike MacGregor,Compass Florida LLC,Kevin Spina,The Keyes Company (PBG),Troy Maleyko,Illustrated Properties LLC (Jupiter),None,None,4,blue,Palm Beach Shores at 2323 Lake Dr Unit 302 in ...
4,PAST SALE,2025-03-07,Condo/Co-op,1400 S Ocean Blvd #604,Boca Raton,FL,33432.0,2519400.0,2.0,2.5,Addison ON The Ocean IN Boca Raton Condo,2156.0,NaN,1985.0,NaN,1169.0,3491.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,Beaches MLS,RX-11018243,N,Y,26.331256,-80.074321,https://www.redfin.com/FL/Boca-Raton/1400-S-Oc...,David Roberts,Sanctuary Realty LLC,Kelly Brooks,Sanctuary Realty LLC,Bonnie Heatzig,Compass Florida LLC,Mary Mangines,Compass Florida LLC,5,blue,Addison ON The Ocean IN Boca Raton Condo at 14...
5,PAST SALE,2025-03-07,Condo/Co-op,550 S Ocean Blvd #401,Boca Raton,FL,33432.0,1580000.0,2.0,2.0,Chalfonte Condo,1479.0,NaN,1974.0,NaN,1068.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11026297,N,Y,26.344431,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Kim Schlinck,Lang Realty/BR,None,None,6,blue,Chalfonte Condo at 550 S Ocean Blvd #401 in Bo...
6,PAST SALE,2025-03-04,Condo/Co-op,2660 S Ocean Blvd Unit 702n,Palm Beach,FL,33480.0,1500000.0,2.0,2.5,Beach Point Condo,1900.0,NaN,1979.0,NaN,789.0,2308.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-10975482,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Cathy Casella,Brown Harris Stevens of PB,Lauren Sbordone,Brown Harris Stevens of PB,Cathy Casella,Brown Harris Stevens of PB,None,None,7,blue,Beach Point Condo at 2660 S Ocean Blvd Unit 70...
7,PAST SALE,2025-03-03,Condo/Co-op,500 SE Mizner Blvd U

## Time on Market Calculator

In [101]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 3, 27) ## List (Earlier) date
date2 = datetime(2025, 3, 3) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

707


## Clean CSV for Datawrapper Chart

In [89]:
chart_df = df_top_ten

In [90]:
chart_df = chart_df.fillna(" ")

In [91]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [92]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [93]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [94]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [95]:
chart_df['$/SQUARE FEET'].astype(float)

0    1155.0
1    1214.0
2    1153.0
3    1313.0
4    1169.0
5    1068.0
6     789.0
7     728.0
8     759.0
9     386.0
Name: $/SQUARE FEET, dtype: float64

In [96]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [97]:
csv_date_string = today.strftime("%m_%d_%Y")

In [98]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [99]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-03-07,2494 S Ocean Blvd Unit L-3 in Boca Raton,"$5,450,000",4,4.5,"4,718",1995,"$1,155",Beaches MLS,RX-11042965,Michael Wells Premier Estate Properties Inc Sc...,Catherine Kohn One World Properties
1,2025-03-03,622 N Flagler Dr #803 in West Palm Beach,"$4,900,000",3,3.5,"4,035",2003,"$1,214",Beaches MLS,RX-10876816,"Margit Brandt Premier Estate Properties, Inc",Adam Levitt The Corcoran Group
2,2025-03-03,3730 N Ocean 4b Dr Unit 4b in Riviera Beach,"$4,200,000",4,4.5,"3,642",2018,"$1,153",Beaches MLS,RX-11048530,"Brian Jacobs Network Property Group, LLC","Brian Jacobs Network Property Group, LLC"
3,2025-03-07,2323 Lake Dr Unit 302 in Singer Island,"$3,400,000",3,3.5,"2,589",2024,"$1,313",Beaches MLS,RX-11004952,Meike MacGregor Compass Florida LLC Kevin Spin...,Troy Maleyko Illustrated Properties LLC (Jupit...
4,2025-03-07,1400 S Ocean Blvd #604 in Boca Raton,"$2,519,400",2,2.5,"2,156",1985,"$1,169",Beaches MLS,RX-11018243,David Roberts Sanctuary Realty LLC Kelly Brook...,Bonnie Heatzig Compass Florida LLC Mary Mangin...
5,2025-03-07,550 S Ocean Blvd #401 in Boca Raton,"$1,580,000",2,2.0,"1,479",1974,"$1,068",Beaches MLS,RX-11026297,Ana Londono Lang Realty/BR,Kim Schlinck Lang Realty/BR
6,2025-03-04,2660 S Ocean Blvd Unit 702n in Palm Beach,"$1,500,000",2,2.5,"1,900",1979,$789,Beaches MLS,RX-10975482,Cathy Casella Brown Harris Stevens of PB Laure...,Cathy Casella Brown Harris Stevens of PB
7,2025-03-03,500 SE Mizner Blvd Unit A902 in Boca Raton,"$1,460,000",2,2.5,"2,005",1998,$728,Beaches MLS,RX-11040277,"Caryn Nadler Premier Estate Properties, Inc","Caryn Nadler Premier Estate Properties, Inc"
8,2025-03-03,2917 S Ocean Blvd #1002 in Highland Beach,"$1,325,000",2,2.0,"1,745",1980,$759,Beaches MLS,RX-11043172,Torm Martinson Campbell&Rosemurgy Real Estate,Eric Levy Coldwell Banker
9,2025-03-05,362 Chambord Ter in Palm Beach Gardens,"$1,140,000",2,2.5,"2,953",2005,$386,Beaches MLS,RX-11051333,Sylvia Isaacs Echo Fine Properties Jeffrey Lic...,Chris Burke Echo Fine Properties Jeffrey Licht...
